In [1]:
from guidance import models, gen, any_char, any_char_but, regex, substring, substring_no_empty, with_temperature
import os
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

In [3]:
# This is the name of the model deployed, such as 'gpt-4' or 'gpt-3.5-turbo
model = os.getenv("AZUREAI_CHAT_MODEL", "Please set the model")

# This is the deployment URL, as provided in the Azure AI playground ('view code'):
azure_endpoint = os.getenv("AZUREAI_CHAT_ENDPOINT", "Please set the endpoint")

# The environment variable should be set to the API key from the Azure AI playground:
api_key=os.getenv("AZUREAI_CHAT_KEY", "Please set API key")


In [4]:
azure_endpoint

'https://gemini-compete-si.openai.azure.com/openai/deployments/GeminiSouthIndiaGPT4/chat/completions?api-version=2024-02-15-preview'

In [5]:
aoai = models.AzureOpenAI(
    model=model,
    azure_endpoint=azure_endpoint,
    api_key=api_key)

# Creative story writing

In [16]:
creative = aoai + """# Instructions
You are a highly creative storyteller who has been tasked with writing a short story for a children's book. The story should include a butterfly, an elephant, and a rainbow. It should teach a moral lesson and have a happy ending.

# Story
"""
from guidance import system, user, assistant

with system():
    lm = model + "You are a highly creative storyteller."
    
with user():
    lm += "You are a highly creative storyteller who has been tasked with writing a short story for a children's book. The story should include a butterfly, an elephant, and a rainbow. It should teach a moral lesson and have a happy ending."

with assistant():
    lm += gen(max_tokens=250, temperature=0.9)

start gen(name="None")
finish gen


In [15]:
lm_str = str(lm)

In [10]:
lm_copy = models.AzureOpenAI(model=model, azure_endpoint=azure_endpoint, api_key=api_key) + lm_str

ValueError: The OpenAI model gpt-4 is a Chat-based model and requires role tags in the prompt!             Make sure you are using guidance context managers like `with system():`, `with user():` and `with assistant():`             to appropriately format your guidance program for this type of model.

: 

In [20]:
with user():
    new_story = lm_copy + "can you make it about a fox and a dog now?"
with assistant():
    new_story += gen(max_tokens=50, temperature=0.9)

In [31]:
creative2 = mistral + """# Instructions
You are a highly creative storyteller who has been tasked with writing a short story for a children's book. The story should include a butterfly, an elephant, and a rainbow. It should teach a moral lesson and have a happy ending.

# Story
Once upon a time, there was a butterfly named Rosie. She was very small, but she had a big dream of flying through the sky. Rosie lived in a city where there were lots of buildings and traffic. It was very crowded, and she felt like she was just one tiny butterfly in a big world.

One day, Rosie saw an elephant named Sam. Sam was very big and strong, and he was carrying a basket of beautiful flowers. She was curious about what he was doing and asked him.

Sam said, "I'm on my way to a parade. I'll be carrying these flowers as part of the celebration."

Rosie was excited to see the parade, but she was worried about her small size. "I'm too little to join the parade," she said.

Sam replied, "Don't worry, Rosie. We can't all be big and strong like me, but we can all make a difference in our own ways. And who knows, maybe someday you'll be able to fly in a parade and show the world your beauty."

Later that day, Rosie went home and saw the parade on TV. She saw Sam carrying the flowers, just as he had promised. She was amazed at his strength and determination.

The next day, Rosie woke up feeling inspired. She went outside and spread her wings. She took flight and soared through the air, feeling the wind against her wings. She flew higher and higher until she reached the top of a skyscraper, where she could see the entire city below her. She felt proud of her accomplishment and knew that she could achieve anything she set her mind to.

Rosie continued to practice her flying and soon became an expert. She would fly through the city, enjoying the sights and sounds. One day, she saw Sam carrying a basket of flowers again. She decided to join him in the parade, and together they marched through the streets, showcasing their talents and bringing joy to the people of the city.

The parade was a success, and Rosie and Sam were both proud of their accomplishments. They continued to work together, inspired by their shared dreams and the belief that everyone, no matter how big or small, can make a difference.

Together they spread the message of hope and perseverance, and their story became a symbol of the power of determination and teamwork. And from then on, whenever there was a parade in the city, there was always a butterfly and an elephant marching side by side."""

## Lipogram
https://en.wikipedia.org/wiki/Gadsby_(novel)

https://en.wikipedia.org/wiki/Lipogram

In [6]:
story_no_e = creative + any_char_but('e')
for i in range(250):
    story_no_e += any_char_but('e')

story_no_e

In [7]:
story_no_a = creative + any_char_but('a')
for i in range(250):
    story_no_a += any_char_but('a')

story_no_a

In [8]:
# Does substring also work?
story_substr = creative + substring_no_empty('afaenijeksilebben')
for i in range(250):
    story_substr += substring_no_empty('afaenijeksilebben')

story_substr

In [9]:
# Define a set of words to use in the story
words = "Once upon a time there was a curious butterfly who loved to explore the world One day the butterfly met a wise old elephant near a beautiful rainbow The elephant taught the butterfly an important lesson about courage and friendship The end"

# Generate the story using only substrings of the predefined words
story = creative + substring(words)
for _ in range(10):  # Adjust the number of generation steps as needed
    story += substring(words)

print(story)

# Instructions
You are a highly creative storyteller who has been tasked with writing a short story for a children's book. The story should include a butterfly, an elephant, and a rainbow. It should teach a moral lesson and have a happy ending.

# Story
Once upon a time


### Trying a different metaprompt

In [10]:
creative_lipogram = mistral + """# Instructions
You are a highly creative storyteller who has been tasked with writing a short story for a children's book. The story should include a butterfly, an elephant, and a rainbow. It should teach a moral lesson and have a happy ending.

Additionally, your story must be a lipogram. A lipogram is a kind of constrained writing or word game consisting of writing paragraphs or longer works in which a particular letter or group of letters is avoided.

In this case, your story must avoid the letter 'e'.

# Story
"""

In [11]:
story_no_e2 = creative_lipogram + with_temperature(any_char_but('e'), temperature=0.5)
for i in range(250):
    story_no_e2 += with_temperature(any_char_but('e'), temperature=0.5)

story_no_e2

In [14]:
no_a = mistral + f"""# Instructions
You are a highly creative storyteller who has been tasked with writing a short story for a children's book. The story should include a butterfly, an elephant, and a rainbow. It should teach a moral lesson and have a happy ending.

Additionally, your story must be a lipogram. A lipogram is a kind of constrained writing or word game consisting of writing paragraphs or longer works in which a particular letter or group of letters is avoided.

In this case, your story must avoid the letter 'a'.

# Story
"""

for i in range(1000):
    no_a += with_temperature(any_char_but('a'), temperature=0.5)

KeyboardInterrupt: 

In [15]:
no_a2 = mistral + f"""# Instructions
Write a short story for a children's book that includes a butterfly, an elephant, and a rainbow. The story should teach a moral lesson and have a happy ending.

The story must be a lipogram, which means you must avoid using the letter 'a' entirely. To achieve this, you may use creative spellings or alternative words that convey the same meaning without using the letter 'a'. For example, instead of 'elephant', you could use 'jumbo' or 'hefty beest'.

Your story should be coherent and engaging, despite the constraint of not using the letter 'a'. Feel free to be as imaginative as possible with your word choices and storytelling.

# Story
"""

for i in range(500):
    no_a2 += with_temperature(any_char_but('Aa'), temperature=0.7)

In [16]:
no_a2 = mistral + f"""# Instructions
Write a short story for a children's book that includes a butterfly, a flower, and a rainbow. The story should teach a moral lesson and have a happy ending.

The story must be a lipogram, which means you must avoid using the letter 'a' entirely. To achieve this, you may use creative spellings or alternative words that convey the same meaning without using the letter 'a'. For example, instead of 'elephant', you could use 'jumbo' or 'hefty beest'.

Your story should be coherent and engaging, despite the constraint of not using the letter 'a'. Feel free to be as imaginative as possible with your word choices and storytelling.

# Story
"""

for i in range(500):
  if i % 2 == 0:
    no_a2 += with_temperature(any_char_but('Aa'), temperature=0.7)
  else:
    no_a2 += gen(max_tokens=2, temperature=0.9)

In [17]:
no_e_base = mistral + f"""# Instructions
Write a short story for a children's book that includes a butterfly, an elephant, and a rainbow. The story should teach a moral lesson and have a happy ending.

The story must be a lipogram, which means you must avoid using the letter 'e' entirely. To achieve this, you may use creative spellings or alternative words that convey the same meaning without using the letter 'e'. For example, instead of 'elephant', you could use 'jumbo' or 'alaphant'.

Your story should be coherent and engaging, despite the constraint of not using the letter 'e'. Feel free to be as imaginative as possible with your word choices and storytelling.

# Story
"""


In [18]:
no_e_re = no_e_base + with_temperature(regex(r'[^e]'), temperature=0.6)
for i in range(250):
    no_e_re += with_temperature(regex(r'[^e]'), temperature=0.6)

In [19]:
for i in range(500):
    no_e_re += with_temperature(regex(r'[^e]'), temperature=0.6)

In [21]:

no_e_re2 = no_e_base + with_temperature(regex(r'[^e]+'), temperature=0.9)
for i in range(250):
    no_e_re2 += with_temperature(regex(r'[^e]+'), temperature=0.9)

Exception: Attempted to use a context length of 4096 tokens, but this LlamaCpp model is only configured to support up to 4096!

In [23]:
no_e_re3 = no_e_base + with_temperature(regex(r'[^eE]+'), temperature=0.7)
for i in range(200):
    no_e_re3 += with_temperature(regex(r'[^eE]+'), temperature=0.7)

Exception: Attempted to use a context length of 4096 tokens, but this LlamaCpp model is only configured to support up to 4096!

In [ ]:
no_e_re4 = no_e_base + with_temperature(regex(r'[^eE]+'), temperature=0.7)
for i in range(250):
  if i % 3 == 0:
    no_e_re4 += with_temperature(regex(r'[^eE]+'), temperature=0.7)
  else:
    no_e_re4 += with_temperature(any_char_but('eE'), temperature=0.9)

KeyboardInterrupt: 

In [24]:
no_e_re5 = no_e_base + with_temperature(regex(r'[^eE]+'), temperature=0.9)
for i in range(250):
  if i % 3 == 0 or i % 5 == 0:
    no_e_re5 += with_temperature(regex(r'[^eE]+'), temperature=0.9)
  else:
    no_e_re5 += with_temperature(any_char_but('eE'), temperature=0.9)

In [ ]:

no_e_re6 = no_e_base + with_temperature(regex(r'[^eE]+'), temperature=0.9)
for i in range(250):
  if i % 3 == 0 or i % 5 == 0:
    no_e_re6 += with_temperature(regex(r'[^eE]+'), temperature=0.9)
  else:
    no_e_re6 += with_temperature(any_char_but('eE'), temperature=0.9)